In [1]:
import os
import random
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import collections as col
from pathlib import Path
from requests_html import HTMLSession
from typing import List, Dict

## Data preparation

In [2]:
session = HTMLSession()

In [3]:
data_path = "./bed-time-stories.txt"
file = Path(data_path)

In [4]:
def crawl_data(_from: int, _to: int) -> Dict[str, List[str]]:
    if file.exists():
        return

    with open(data_path, "w") as outfile:
        for page in range(_from, _to + 1):
            r = session.get(f"http://truyencotich.vn/truyen-co-tich/co-tich-viet-nam/page/{page}")

            for link in set([element.attrs["href"] for element in r.html.find("article a")]):
                page = session.get(link)

                try:
                    content = page.html.find("#content .entry-content", first=True).text
                
                    outfile.write(content)
                except:
                    pass

In [5]:
crawl_data(1, 30)

## Functions

In [6]:
def vocab_encode(text, vocab):
    return [vocab.index(x) + 1 for x in text if x in vocab]

def vocab_decode(array, vocab):
    return ''.join([vocab[x - 1] for x in array])

def read_data(filename, vocab, window, overlap):
    lines = [line.strip() for line in open(filename, 'r').readlines()]
    while True:
        random.shuffle(lines)

        for text in lines:
            text = vocab_encode(text, vocab)
            for start in range(0, len(text) - window, overlap):
                chunk = text[start: start + window]
                chunk += [0] * (window - len(chunk))
                yield chunk

def read_batch(stream, batch_size):
    batch = []
    for element in stream:
        batch.append(element)
        if len(batch) == batch_size:
            yield batch
            batch = []
    yield batch

In [7]:
vocal = "\n !'()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVXY[]abcdeghijklmnopqrstuvxyz\xa0«»ÀÁÂÉÊÍÐÒÓÔÚÜÝàáâãèéêìíòóôõøùúýĂăĐđĩŒũƠơƯự̀́̃̉ạẢảẤấầẩẫậắằẳẵặẹẻẽẾếềểễệỈỉỊịọỏỐốỒồổỗộỚớỜờỞởỡợụỦủứỪừửữựỳỵỷỹ–‘’“”…″金"

In [8]:
class CharRNN(object):
    def __init__(self, model):
        self.model = model
        self.path = model + '.txt'
        self.vocab = vocal
        self.seq = tf.placeholder(tf.int32, [None, None])
        self.temp = tf.constant(1.5)
        self.hidden_sizes = [128, 256]
        self.batch_size = 64
        self.lr = 0.01
        self.example_to_run = 10
        self.skip_step = 10
        self.num_steps = 300 # for RNN unrolled
        self.len_generated = 2000
        self.gstep = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

    def create_rnn(self, seq):
        layers = [tf.nn.rnn_cell.GRUCell(size) for size in self.hidden_sizes]
        cells = tf.nn.rnn_cell.MultiRNNCell(layers)
        batch = tf.shape(seq)[0]
        zero_states = cells.zero_state(batch, dtype=tf.float32)
        self.in_state = tuple([tf.placeholder_with_default(state, [None, state.shape[1]]) 
                                for state in zero_states])
        # this line to calculate the real length of seq
        # all seq are padded to be of the same length, which is num_steps
        length = tf.reduce_sum(tf.reduce_max(tf.sign(seq), 2), 1)
        self.output, self.out_state = tf.nn.dynamic_rnn(cells, seq, length, self.in_state)

    def create_model(self):
        seq = tf.one_hot(self.seq, len(self.vocab))
        self.create_rnn(seq)
        self.logits = tf.layers.dense(self.output, len(self.vocab), None)
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits[:, :-1], 
                                                        labels=seq[:, 1:])
        self.loss = tf.reduce_sum(loss)
        # sample the next character from Maxwell-Boltzmann Distribution 
        # with temperature temp. It works equally well without tf.exp
        self.sample = tf.multinomial(tf.exp(self.logits[:, -1] / self.temp), 1)[:, 0] 
        self.opt = tf.train.AdamOptimizer(self.lr).minimize(self.loss, global_step=self.gstep)

    def train(self):
        saver = tf.train.Saver()
        start = time.time()
        min_loss = None

        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)

        with tf.Session(config = tf.ConfigProto(gpu_options=gpu_options)) as sess:
            sess.run(tf.global_variables_initializer())
            
            ckpt = tf.train.get_checkpoint_state(os.path.dirname('models/' + self.model + '/checkpoint'))
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
            
            iteration = self.gstep.eval()
            stream = read_data(self.path, self.vocab, self.num_steps, overlap=self.num_steps // 2)
            data = read_batch(stream, self.batch_size)

            for i in range(self.example_to_run):
                batch = next(data)
                
                # for batch in read_batch(read_data(DATA_PATH, vocab)):
                batch_loss, _ = sess.run([self.loss, self.opt], {self.seq: batch})
                
                if (iteration + 1) % self.skip_step == 0:
                    print('Iter {}. \n    Loss {}. Time {}'.format(iteration, batch_loss, time.time() - start))
                    self.online_infer(sess)
                    start = time.time()
#                     checkpoint_name = 'models/' + self.model + '/char-rnn'
#                     if min_loss is None:
#                         saver.save(sess, checkpoint_name, iteration)
#                     elif batch_loss < min_loss:
#                         saver.save(sess, checkpoint_name, iteration)
#                         min_loss = batch_loss
                iteration += 1

    def online_infer(self, sess):
        """ Generate sequence one character at a time, based on the previous character
        """
        for seed in ["Ngày"]:
            sentence = seed
            state = None
            for _ in range(self.len_generated):
                batch = [vocab_encode(sentence[-1], self.vocab)]
                feed = {self.seq: batch}
                if state is not None: # for the first decoder step, the state is None
                    for i in range(len(state)):
                        feed.update({self.in_state[i]: state[i]})
                index, state = sess.run([self.sample, self.out_state], feed)
                sentence += vocab_decode(index, self.vocab)
            print('\t' + sentence)

In [9]:
model = 'bed-time-stories'

lm = CharRNN(model)
lm.create_model()
lm.train()

Ngày hôm sau không nén được giận, nó liền bứt con ốc ra và vứt luôn xuống ao. Ngực nó bỗng đau nhói lên ở chỗ con ốc hay nằm trước kia, và liền đó, nó ngã lăn ra, hộc máu chết ngay tức khắc. Lão nhà giàu vừa sợ, vừa giận, liền cho một tên tay sai đi gọi thầy tới nhà để nó hỏi tội. Tên tay sai tới nh





 ngã lăn ra, hộc máu chết ngay tức khắc. Lão nhà giàu vừa sợ, vừa giận, liền cho một tên tay sai đi gọi thầy tới nhà để nó hỏi tội. Tên tay sai tới nhà thì vừa gặp lúc ông thầy đang nói chuyện với hai anh học trò nghèo. Vốn là một tay ranh ma, xảo quyệt, nó liền nấp vào một chỗ kín và rình nghe. Nó 





Một ngày nọ, có một chàng trai tuấn tú lên Long đang trên đường đi thi hội, tới đây thì lỡ độ đường, trời cũng đã bắt đầu sẩm tối. Thấy có ngôi nhà đẹp, anh ghé vào xin nghỉ nhờ một đêm. Thấy có người xin ngủ nhờ, hai vợ chồng phú ông kể lại cho Long câu chuyện mà gia đình ông đang gặp phải, kể xong





Nhưng không may vụ hạ năm ấy mất mùa. Rồi tiếp đến vụ mùa nắng hạn nên cũng chẳng 